In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("hw7.ipynb")

# CPSC 330 - Applied Machine Learning 

## Homework 7: Word embeddings and topic modeling 
**Due date: See the [Calendar](https://calendar.google.com/calendar/u/0/embed?src=7a04205ae91b85e82ebc74daddbf2933c6b6723b81abb966f0e69c66a996c43b@group.calendar.google.com&ctz=America/Vancouver&pli=1).**

## Imports

In [2]:
import os
from hashlib import sha1

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline

<br><br>

## Submission instructions
<hr>
rubric={points:2}

You will receive marks for correctly submitting this assignment. To submit this assignment, follow the instructions below:

- **You may work on this assignment in a group (group size <= 4) and submit your assignment as a group.** 
- Below are some instructions on working as a group.  
    - The maximum group size is 4. 
    - You can choose your own group members. 
    - Use group work as an opportunity to collaborate and learn new things from each other. 
    - Be respectful to each other and make sure you understand all the concepts in the assignment well. 
    - It's your responsibility to make sure that the assignment is submitted by one of the group members before the deadline. [Here](https://help.gradescope.com/article/m5qz2xsnjy-student-add-group-members) are some instructions on adding group members in Gradescope.  
- Upload the .ipynb file to Gradescope.
- **If the .ipynb file is too big or doesn't render on Gradescope for some reason, also upload a pdf or html in addition to the .ipynb.** 
- Make sure that your plots/output are rendered properly in Gradescope.

<br><br><br><br>

## Exercise 1:  Exploring pre-trained word embeddings <a name="1"></a>
<hr>

In lecture 18, we talked about natural language processing (NLP). Using pre-trained word embeddings is very common in NLP. It has been shown that pre-trained word embeddings work well on a variety of text classification tasks. These embeddings are created by training a model like Word2Vec on a huge corpus of text such as a dump of Wikipedia or a dump of the web crawl. 

A number of pre-trained word embeddings are available out there. Some popular ones are: 

- [GloVe](https://nlp.stanford.edu/projects/glove/)
    * trained using [the GloVe algorithm](https://nlp.stanford.edu/pubs/glove.pdf) 
    * published by Stanford University 
- [fastText pre-trained embeddings for 294 languages](https://fasttext.cc/docs/en/pretrained-vectors.html) 
    * trained using the fastText algorithm
    * published by Facebook
    
In this exercise, you will be exploring GloVe Wikipedia pre-trained embeddings. The code below loads the word vectors trained on Wikipedia using an algorithm called Glove. You'll need `gensim` package in your cpsc330 conda environment to run the code below. 

```
> conda activate cpsc330
> conda install -c anaconda gensim
```

In [3]:
import gensim
import gensim.downloader

print(list(gensim.downloader.info()["models"].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [4]:
# This will take a while to run when you run it for the first time.
import gensim.downloader as api

glove_wiki_vectors = api.load("glove-wiki-gigaword-100")

In [5]:
len(glove_wiki_vectors)

400000

There are 400,000 word vectors in this pre-trained model. 

Now that we have GloVe Wiki vectors loaded in `glove_wiki_vectors`, let's explore the embeddings. 

<br><br>

<!-- BEGIN QUESTION -->

### 1.1 Word similarity using pre-trained embeddings
rubric={points:2}

**Your tasks:**

- Come up with a list of 4 words of your choice and find similar words to these words in `glove_wiki_vectors` embeddings. 

<div class="alert alert-warning">

Solution_1.1
    
</div>

_Points:_ 2

My words will be: click, earthquake, cow, and happy

In [6]:
# Some code inspired from Lecture 17 Notes
words = ["click", "earthquake", "cow", "sad"]

for word in words:
    sim_word = glove_wiki_vectors.most_similar(word)
    print(f"The words most similar to '{word}' are {sim_word}")

The words most similar to 'click' are [('clicking', 0.7805083990097046), ('clicks', 0.6462287306785583), ('lets', 0.6371946334838867), ('browse', 0.6323150992393494), ('clicked', 0.6275697946548462), ('delete', 0.6217952370643616), ('button', 0.6146154999732971), ('user', 0.6108575463294983), ('download', 0.5969396829605103), ('tab', 0.591617226600647)]
The words most similar to 'earthquake' are [('quake', 0.9480070471763611), ('tsunami', 0.8075350522994995), ('temblor', 0.7902826070785522), ('magnitude', 0.7707369923591614), ('earthquakes', 0.7257689237594604), ('disaster', 0.7229208946228027), ('aftershock', 0.7178042531013489), ('tremor', 0.7131491303443909), ('tsunamis', 0.7090721726417542), ('quakes', 0.7071856260299683)]
The words most similar to 'cow' are [('cows', 0.754188597202301), ('pig', 0.7517361044883728), ('mad', 0.7499966621398926), ('sheep', 0.7310901284217834), ('pigs', 0.7278857827186584), ('bovine', 0.7249352931976318), ('cattle', 0.7192909121513367), ('goat', 0.704

<!-- END QUESTION -->

<br><br>

### 1.2 Word similarity using pre-trained embeddings
rubric={autograde:2}

**Your tasks:**
1. Calculate cosine similarity for the following word pairs (`word_pairs`) using the [`similarity`](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) method of `glove_wiki_vectors`.

In [7]:
word_pairs = [
    ("coast", "shore"),
    ("clothes", "closet"),
    ("old", "new"),
    ("smart", "intelligent"),
    ("dog", "cat"),
    ("tree", "lawyer"),
]

<div class="alert alert-warning">

Solution_1.2
    
</div>

_Points:_ 2

In [8]:
# Some code inspired from Lecture 17 Notes

coast_shore_similarity = glove_wiki_vectors.similarity("coast", "shore")
clothes_closet_similarity = glove_wiki_vectors.similarity("clothes", "closet")
old_new_similarity = glove_wiki_vectors.similarity("old", "new")
smart_intelligent_similarity = glove_wiki_vectors.similarity("smart", "intelligent")
dog_cat_similarity = glove_wiki_vectors.similarity("dog", "cat")
tree_lawyer_similarity = glove_wiki_vectors.similarity("tree", "lawyer")


In [9]:
grader.check("q1.2")

q1.2 results: All test cases passed!

<br><br>

<!-- BEGIN QUESTION -->

### 1.3 Stereotypes and biases in embeddings
rubric={points:6}

Word vectors contain lots of useful information. But they also contain stereotypes and biases of the texts they were trained on. In the lecture, we saw an example of gender bias in Google News word embeddings. Here we are using pre-trained embeddings trained on Wikipedia data. 

**Your tasks:**

1. Explore whether there are any worrisome biases or stereotypes present in these embeddings or not by trying out at least 4 examples. You can use the following two methods or other methods of your choice to explore what kind of stereotypes and biases are encoded in these embeddings. 
    - the `analogy` function below which gives word analogies (an example shown below)
    - [similarity](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) or [distance](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=distance#gensim.models.keyedvectors.KeyedVectors.distances) methods (an example is shown below)   
2. Discuss your observations.

> Note that most of the recent embeddings are de-biased. But you might still observe some biases in them. Also, not all stereotypes present in pre-trained embeddings are necessarily bad. But you should be aware of them when you use them in your models. 

An example of using analogy to explore biases and stereotypes.

In [10]:
def analogy(word1, word2, word3, model=glove_wiki_vectors):
    """
    Returns analogy word using the given model.

    Parameters
    --------------
    word1 : (str)
        word1 in the analogy relation
    word2 : (str)
        word2 in the analogy relation
    word3 : (str)
        word3 in the analogy relation
    model :
        word embedding model

    Returns
    ---------------
        pd.dataframe
    """
    print("%s : %s :: %s : ?" % (word1, word2, word3))
    sim_words = model.most_similar(positive=[word3, word2], negative=[word1])
    return pd.DataFrame(sim_words, columns=["Analogy word", "Score"])

In [11]:
analogy("man", "doctor", "woman")

man : doctor :: woman : ?


,Analogy word,Score
0,nurse,0.773523
1,physician,0.718943
2,doctors,0.682433
3,patient,0.675068
4,dentist,0.672603
5,pregnant,0.664246
6,medical,0.652045
7,nursing,0.645348
8,mother,0.639333
9,hospital,0.638750


An example of using similarity between words to explore biases and stereotypes.  

In [12]:
glove_wiki_vectors.similarity("white", "rich")

0.447236

In [13]:
glove_wiki_vectors.similarity("black", "rich")

0.5174519

<div class="alert alert-warning">

Solution_1.3
    
</div>

_Points:_ 6

In [14]:
analogy("person", "job", "asian")

person : job :: asian : ?


,Analogy word,Score
0,economic,0.607589
1,asia,0.585124
2,boost,0.584264
3,markets,0.571742
4,financial,0.564098
5,economy,0.553295
6,amid,0.550781
7,slump,0.549783
8,jobs,0.548123
9,global,0.547367


In [15]:
analogy("basketball", "shot", "soccer")

basketball : shot :: soccer : ?


,Analogy word,Score
0,kick,0.652865
1,minute,0.635575
2,off,0.635209
3,shots,0.630597
4,attack,0.620873
5,shoot,0.619040
6,caught,0.612677
7,fired,0.610697
8,turned,0.609696
9,killed,0.609647


In [16]:
glove_wiki_vectors.similarity("male", "competent")

0.295371

In [17]:
glove_wiki_vectors.similarity("female", "competent")

0.33420974

In [18]:
glove_wiki_vectors.similarity("man", "smart")

0.4778229

In [19]:
glove_wiki_vectors.similarity("woman", "smart")

0.36801586

In [20]:
glove_wiki_vectors.similarity("cookie", "brownie")

0.63438654

There are a few observations that we can make. On the one hand, it seems as though the model can make good connections between words that are expected to be similar (ex: cookie and brownie, basketball is to shot as soccer is to kick, etc).

When trying to see if there may be some worrying biases, it's difficult to tell. While female:competent similarity was higher than male:competent similarity, man:smart similarity was higher than woman:smart similarity. From such a short sample, it's difficult to see whether or not there is biases, so this should be taken into account.

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.4 Discussion
rubric={points:4}

**Your tasks:**
1. Based on your exploration above, comment on the overall quality of these pre-trained embeddings. 
2. In the lecture, we saw that our pre-trained word embedding model output an analogy that reinforced a gender stereotype. Give an example of how using such a model could cause harm in the real world.

<div class="alert alert-warning">

Solution_1.4
    
</div>

_Points:_ 4

1. The overall quality of the pre-trained embeddings seems to be relatively de-biased. It was able to find some useful patterns, and they didn't seem too worrisome.
2. Results from these models are often presented as research findings to the public, which may be used to enact policies. If the models reinforce these stereotypes, the resulting policies may not be effective, or worse, could be harmful. 

<!-- END QUESTION -->

<br><br>

### 1.5 Classification with pre-trained embeddings 
rubric={autograde:8}

In lecture 18, we saw that you can conveniently get word vectors with `spaCy` with `en_core_web_md` model. In this exercise, you'll use word embeddings in multi-class text classification task. We will use [HappyDB](https://www.kaggle.com/ritresearch/happydb) corpus which contains about 100,000 happy moments classified into 7 categories: *affection, exercise, bonding, nature, leisure, achievement, enjoy_the_moment*. The data was crowd-sourced via [Amazon Mechanical Turk](https://www.mturk.com/). The ground truth label is not available for all examples, and in this lab, we'll only use the examples where ground truth is available (~15,000 examples). 

- Download the data from [here](https://www.kaggle.com/ritresearch/happydb).
- Unzip the file and copy it under the data/ directory in this homework directory.

The code below reads the data CSV (assuming that it's present in the current directory as *cleaned_hm.csv*),  cleans it up a bit, and splits it into train and test splits. 

**Your tasks:**

1. Train logistic regression with bag-of-words features (`CountVectorizer`, be sure to filter out stop words), calculate the train and test accuracy and show a [classification report](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html) on the test set.
2. Train logistic regression with average embedding representation extracted using spaCy, calculate the train and test accuracy and show [classification report](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html) on the test set. (You can refer to lecture 18 notes for this.)  

In [21]:
df = pd.read_csv("data/cleaned_hm.csv", index_col=0)
sample_df = df.dropna()
sample_df.head()

,wid,reflection_period,original_hm,cleaned_hm,modified,num_sentence,ground_truth_category,predicted_category
hmid,,,,,,,,
27676,206,24h,We had a serious talk with some friends of our...,We had a serious talk with some friends of our...,True,2,bonding,bonding
27678,45,24h,I meditated last night.,I meditated last night.,True,1,leisure,leisure
27697,498,24h,My grandmother start to walk from the bed afte...,My grandmother start to walk from the bed afte...,True,1,affection,affection
27705,5732,24h,I picked my daughter up from the airport and w...,I picked my daughter up from the airport and w...,True,1,bonding,affection
27715,2272,24h,when i received flowers from my best friend,when i received flowers from my best friend,True,1,bonding,bonding


In [22]:
sample_df = sample_df.rename(
    columns={"cleaned_hm": "moment", "ground_truth_category": "target"}
)

In [23]:
train_df, test_df = train_test_split(sample_df, test_size=0.3, random_state=123)
X_train, y_train = train_df["moment"], train_df["target"]
X_test, y_test = test_df["moment"], test_df["target"]

You need `spacy` to run the code below. If it's not in your course conda environment, you need to install it. 

> conda install -c conda-forge spacy

You also need to download the following language model. 

> python -m spacy download en_core_web_md

In [24]:
import spacy
import en_core_web_md

# nlp = spacy.load("en_core_web_md")
nlp = en_core_web_md.load()

<div class="alert alert-warning">

Solution_1.5
    
</div>

_Points:_ 8

Train logistic regression with bag-of-words features (CountVectorizer, be sure to filter out stop words), calculate the train and test accuracy and show a classification report on the test set.


Train logistic regression with average embedding representation extracted using spaCy, calculate the train and test accuracy and show classification report on the test set. (You can refer to lecture 18 notes for this.)

In [25]:
# Some code taken from Lecture 17 Notes

bow_pipe = make_pipeline(
    CountVectorizer(stop_words="english"), LogisticRegression(max_iter=1000)
)
bow_pipe.named_steps["countvectorizer"].fit(X_train)
X_train_transformed = bow_pipe.named_steps["countvectorizer"].transform(X_train)
bow_pipe.fit(X_train, y_train);

In [26]:
from sklearn.metrics import classification_report

bow_train_accuracy = bow_pipe.score(X_train, y_train)
bow_test_accuracy = bow_pipe.score(X_test, y_test)


bow_report = classification_report(y_test, bow_pipe.predict(X_test), digits=2) # use digits=2

In [27]:

X_train_embeddings = pd.DataFrame([text.vector for text in nlp.pipe(X_train)])
X_test_embeddings = pd.DataFrame([text.vector for text in nlp.pipe(X_test)])


In [28]:
X_train_embeddings.shape

(9887, 300)

In [29]:
X_train_embeddings.head()

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,-0.690874,-0.755216,-3.279650,-1.111956,2.333812,1.712701,0.346985,5.052109,-1.078272,1.564646,...,0.978959,0.228331,0.843013,-1.340969,-0.884684,1.971317,-0.880458,-0.599687,-5.299727,-0.139515
1,0.021659,-1.051312,-4.063361,-1.818072,3.027572,-1.226410,0.035204,2.806751,-0.181556,1.530708,...,0.873182,-1.440154,0.219029,-1.426304,-1.591761,-0.475234,-0.122053,0.695197,-4.421231,-0.027261
2,-0.101639,-1.674547,-1.391433,-3.230565,0.636887,0.756545,0.205278,5.351284,-2.872737,3.033967,...,0.594443,0.435623,-0.556652,-2.759545,-1.521208,0.728254,-0.365033,1.230975,-5.989079,0.142662
3,-1.502830,-0.245505,-2.919757,-1.312146,3.315014,-0.281209,1.019417,3.909962,-0.440415,0.797455,...,0.452126,-0.261931,-0.375499,-0.218783,-1.355882,0.384430,0.619531,0.548577,-3.217188,0.119259
4,-0.503838,-0.450076,-4.626111,-2.820389,4.793096,-0.805586,1.518527,5.712772,1.548753,1.589522,...,-0.293538,-0.217686,-3.700689,-1.188137,0.648456,-2.548641,-0.413536,1.096762,-3.927443,-1.008204


In [30]:
lgr = LogisticRegression(max_iter=2000)
lgr.fit(X_train_embeddings, y_train)

emb_train_accuracy = lgr.score(X_train_embeddings, y_train)
emb_test_accuracy = lgr.score(X_test_embeddings, y_test)

emb_report = classification_report(y_test, lgr.predict(X_test_embeddings), digits=2) # use digits=2

C:\Users\Brad\miniconda3\envs\cpsc330\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [31]:
grader.check("q1.5")

q1.5 results:
    q1.5 - 1 result:
        ✅ Test case passed

    q1.5 - 2 result:
        ✅ Test case passed

    q1.5 - 3 result:
        ❌ Test case failed
        Trying:
            assert not emb_train_accuracy is None, "Are you using the correct variable name?"
        Expecting nothing
        ok
        Trying:
            assert not emb_test_accuracy is None, "Are you using the correct variable name?"
        Expecting nothing
        ok
        Trying:
            assert not emb_report is None, "Are you using the correct variable name?"
        Expecting nothing
        ok
        Trying:
            assert (sha1(str(round(emb_train_accuracy, 3)).encode('utf8')).hexdigest() == '1725dc3fe05c0fd60ff7438a035951bf371f980c'), "Did you calculate emb_train_accuracy correctly?"
        Expecting nothing
        **********************************************************************
        Line 5, in q1.5 2
        Failed example:
            assert (sha1(str(round(emb_train_accuracy, 3)).encode('utf8')).hexdigest() == '1725dc3fe05c0fd60ff7438a035951bf371f980c'), "Did you calculate emb_train_accuracy correctly?"
        Exception raised:
            Traceback (most recent call last):
              File "C:\Users\Brad\miniconda3\envs\cpsc330\lib\doctest.py", line 1350, in __run
                exec(compile(example.source, filename, "single",
              File "<doctest q1.5 2[3]>", line 1, in <module>
                assert (sha1(str(round(emb_train_accuracy, 3)).encode('utf8')).hexdigest() == '1725dc3fe05c0fd60ff7438a035951bf371f980c'), "Did you calculate emb_train_accuracy correctly?"
            AssertionError: Did you calculate emb_train_accuracy correctly?
        Trying:
            assert (sha1(str(round(emb_test_accuracy, 3)).encode('utf8')).hexdigest() == 'e15316688f329c21599d094d3698107ff5981f4b'), "Did you calculate emb_test_accuracy correctly?"
        Expecting nothing
        **********************************************************************
        Line 6, in q1.5 2
        Failed example:
            assert (sha1(str(round(emb_test_accuracy, 3)).encode('utf8')).hexdigest() == 'e15316688f329c21599d094d3698107ff5981f4b'), "Did you calculate emb_test_accuracy correctly?"
        Exception raised:
            Traceback (most recent call last):
              File "C:\Users\Brad\miniconda3\envs\cpsc330\lib\doctest.py", line 1350, in __run
                exec(compile(example.source, filename, "single",
              File "<doctest q1.5 2[4]>", line 1, in <module>
                assert (sha1(str(round(emb_test_accuracy, 3)).encode('utf8')).hexdigest() == 'e15316688f329c21599d094d3698107ff5981f4b'), "Did you calculate emb_test_accuracy correctly?"
            AssertionError: Did you calculate emb_test_accuracy correctly?
        Trying:
            assert (sha1(str(''.join(emb_report.split())[:41]).encode('utf8')).hexdigest() == 'c41ae404dc1f2e19d22b9ee598bd53c9c176cf44'), "Did you pass the correct arguments to classification_report?"
        Expecting nothing
        ok

<br><br>

<!-- BEGIN QUESTION -->

### 1.6 Discussion
rubric={points:6}

**Your tasks:**
1. Briefly explain the difference between using `CountVectorizer` vs. average-embedding approach for text classification.  
2. Which representation among these two would be more interpretable? Why?   
3. Are we using any transfer learning here? If yes, are you observing any benefits of transfer learning? Briefly discuss. 

<div class="alert alert-warning">

Solution_1.6
    
</div>

_Points:_ 6

1. While the embedding approach uses pre-trained models that allow us to capture relationships between words (and thus, their meanings), CountVectorizer isn't able to do that. CountVectorizer simply uses the presence/count of word to train the model.


2. I'd say that CountVectorizer is somewhat more interpretable. This is because we can still see that words that are used as features. Using embeddings, the resulting features are arbitrary numbers that are difficult to interpret.

In [32]:
print(f"Count Vectorizer Train Score: {bow_train_accuracy}")
print(f"Count Vectorizer Test Score: {bow_test_accuracy} \n")

print(f"Embeddings Train Score: {emb_train_accuracy}")
print(f"Embeddings Test Score: {emb_test_accuracy}")

Count Vectorizer Train Score: 0.9562051178314959
Count Vectorizer Test Score: 0.8173666823973572 

Embeddings Train Score: 0.8499039142308081
Embeddings Test Score: 0.7937706465313827


3. Yes, average-embedding approach using a pre-trained model is an example of transfer-learning. Looking at the scores above, it seems to be less overfitted than CountVectorizer, but still has a worse test score overall.

<!-- END QUESTION -->

<br><br><br><br>

## Exercise 2: Topic modeling 

The overarching goal of topic modeling is understanding high-level themes in a large collection of texts in an unsupervised way. 

In this exercise you will explore topics in a subset of `scikit-learn`'s [20 newsgroups text dataset](https://scikit-learn.org/0.19/datasets/twenty_newsgroups.html) using `scikit-learn`'s `LatentDirichletAllocation` (LDA) model. 

Usually, topic modeling is used for discovering abstract "topics" that occur in a collection of documents when you do not know the actual topics present in the documents. But 20 newsgroups text dataset is labeled with categories (e.g., sports, hardware, religion), and you will be able to cross-check the topics discovered by your model with these available topics. 

The starter code below loads the train and test portion of the data and convert the train portion into a pandas DataFrame. For speed, we will only consider documents with the following 8 categories. 

In [33]:
from sklearn.datasets import fetch_20newsgroups

In [34]:
cats = [
    "rec.sport.hockey",
    "rec.sport.baseball",
    "soc.religion.christian",
    "alt.atheism",
    "comp.graphics",
    "comp.windows.x",
    "talk.politics.mideast",
    "talk.politics.guns",
]  # We'll only consider these categories out of 20 categories for speed.

newsgroups_train = fetch_20newsgroups(
    subset="train", remove=("headers", "footers", "quotes"), categories=cats
)
X_news_train, y_news_train = newsgroups_train.data, newsgroups_train.target
df = pd.DataFrame(X_news_train, columns=["text"])
df["target"] = y_news_train
df["target_name"] = [
    newsgroups_train.target_names[target] for target in newsgroups_train.target
]
df

,text,target,target_name
0,"You know, I was reading 18 U.S.C. 922 and some...",6,talk.politics.guns
1,\n\n\nIt's not a bad question: I don't have an...,1,comp.graphics
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics
3,"The following problem is really bugging me,\na...",2,comp.windows.x
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast
...,...,...,...
4558,Hi Everyone ::\n\nI am looking for some soft...,1,comp.graphics
4559,Archive-name: x-faq/part3\nLast-modified: 1993...,2,comp.windows.x
4560,"\nThat's nice, but it doesn't answer the quest...",6,talk.politics.guns
4561,"Hi,\n I just got myself a Gateway 4DX-33V ...",2,comp.windows.x


In [35]:
newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.windows.x',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast']

<br><br>

<!-- BEGIN QUESTION -->

### 2.1 Preprocessing using [spaCy](https://spacy.io/)
rubric={points:8}

In this exercise you'll prepare the data for topic modeling using [spaCy](https://spacy.io/). Preprocessing is a crucial step before training an LDA model and it markedly affects topic modeling results. So let's carry out preprocessing. 

**Your tasks:** 

Write code to carry out preprocessing of the "text" column in the dataframe above and store the preprocessed text in a new column called "text_pp" in the dataframe. 


Note that there is no such thing as "perfect" preprocessing. You'll have to make your own judgments and decisions on which tokens are more informative and which ones are less informative for the given task. Some common text preprocessing steps for topic modeling are: 
- getting rid of slashes or other weird characters
- sentence segmentation and tokenization      
- getting rid of urls and email addresses
- getting rid of other fairly unique tokens which are not going to help us in topic modeling  
- excluding stopwords and punctuation 
- lemmatization        

You might have to go back and forth between the preprocessing and topic modeling and interpretation steps in the next exercises. 

> Check out [these available attributes](https://spacy.io/api/token#attributes) for `token` in spaCy which might help you with preprocessing. 

> You can also get rid of words with specific POS tags. [Here](https://spacy.io/api/annotation/#pos-en) is the list of part-of-speech tags used in spaCy. 

> Note that preprocessing the corpus might take some time. So here are a couple of suggestions: 1) During the debugging phase, work on a smaller subset of the data. 2) Once you're done with the preprocessing part, you might want to save the preprocessed data so that you don't run the preprocessing part every time you run the notebook. 


<div class="alert alert-warning">

Solution_2_1
    
</div>

_Points:_ 8

_Type your answer here, replacing this text._

In [36]:
# Some code taken from Lecture 17 Notes

nlp = spacy.load("en_core_web_md", disable=["parser", "ner"])

In [37]:
def preprocess(
    doc,
    min_token_len=2,
    irrelevant_pos=["ADV", "PRON", "CCONJ", "PUNCT", "PART", "DET", "ADP", "SPACE"],
):

    clean_text = []

    for token in doc:
        if (
            token.is_stop == False  # Check if it's not a stopword
            and len(token) > min_token_len  # Check if the word meets minimum threshold
            and token.pos_ not in irrelevant_pos
            and token.like_url == False # Check if it's not an url
            and token.like_email == False # Check if it's not an email
            and token.like_num == False # Check if it's not a number
        ):  # Check if the POS is in the acceptable POS tags
            lemma = token.lemma_  # Take the lemma of the word
            clean_text.append(lemma.lower())
    return " ".join(clean_text)

In [38]:
df["text_pp"] = [preprocess(text) for text in nlp.pipe(df["text"])]

In [39]:
df.head(5)

,text,target,target_name,text_pp
0,"You know, I was reading 18 U.S.C. 922 and some...",6,talk.politics.guns,know read u.s.c. sence wonder help u.s.c. prov...
1,\n\n\nIt's not a bad question: I don't have an...,1,comp.graphics,bad question ref list algorithm think bit hard...
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics,actuallay hand support idea have newsgroup asp...
3,"The following problem is really bugging me,\na...",2,comp.windows.x,follow problem bug appreciate help create wind...
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast,late upi foreign ministry spokesman ferhat ata...


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.2 Justification
rubric={points:2}

**Your tasks:**

Outline the preprocessing steps you carried out in the previous exercise and provide a brief justification for these steps.

<div class="alert alert-warning">

Solution_2_2
    
</div>

_Points:_ 2

We carry out the following preprocessing steps
- excluding stop words: We do this to remove very common words like (prepositions or conjunctions) that are unlikely to be helpful to our model
- exclude tokens that are less than 2 characters: Many short words are not actual words. Things like "uh",  "um" or other typos are not helpful to us
- removed irrelevant parts of speech: There are particular parts of speech that make sense gramatically, but don't actually add much meaning to a sentence. Those can be removed
- Remove numbers: Numbers tend to numerate things and are unlikely to show up multiple times in meaningful ways
- Remove URLs and emails: URLS and emails are meant to be unique and are often very specific. This means they won't provide much value to our model
- Lemmatize and lowercase words: This is to make our model treat different words with the same meaning in the same way. For example, "jump" and "jumped"

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.3 Build a topic model using sklearn's LatentDirichletAllocation
rubric={points:4}

**Your tasks:**
1. Create a topic model on the preprocessed data using [sklearn's `LatentDirichletAllocation`](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html). Pick a reasonable number for `n_components`, i.e., number of topics and briefly justify your choice.

<div class="alert alert-warning">

Solution_2_3
    
</div>

_Points:_ 4

In [40]:
# Some code taken from Lecture 17 Notes

vec = CountVectorizer(stop_words="english")
df_X = vec.fit_transform(df["text_pp"])

In [41]:
from sklearn.decomposition import LatentDirichletAllocation

n_topics = 8 # number of topics

lda = LatentDirichletAllocation(
    n_components=n_topics, learning_method="batch", max_iter=10, random_state=0
)
lda.fit(df_X) 
document_topics = lda.transform(df_X)

I picked n_topics = 8 due to the original 8 targets we saw back in the intro for question 2.

['alt.atheism',
 'comp.graphics',
 'comp.windows.x',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast']

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.4 Exploring word topic association
rubric={points:5}

**Your tasks:**
1. Show top 10 words for each of your topics and suggest labels for each of the topics (similar to how we came up with labels "health and nutrition", "fashion", and "machine learning" in the toy example we saw in class). 

> If your topics do not make much sense, you might have to go back to preprocessing in Exercise 2.1, improve it, and train your LDA model again. 

<div class="alert alert-warning">

Solution_2_4
    
</div>

_Points:_ 5

In [42]:
# Some code taken from Lecture 17 Notes

topic_labels = ['topic_0', 'topic_1', 'topic_2', 'topic_3', 'topic_4', 'topic_5', 'topic_6', 'topic_7']


sorting = np.argsort(lda.components_, axis=1)[:, ::-1]
feature_names = np.array(vec.get_feature_names_out())

In [43]:
import mglearn

mglearn.tools.print_topics(
    topics=range(n_topics),
    feature_names=feature_names,
    sorting=sorting,
    topics_per_chunk=5,
    n_words=10,
)

topic 0       topic 1       topic 2       topic 3       topic 4       
--------      --------      --------      --------      --------      
god           israel        entry         gun           game          
think         armenian      jesus         people        team          
people        turkish       section       weapon        play          
know          jews          god           law           player        
believe       people        son           firearm       season        
say           israeli       use           right         win           
question      armenians     shall         control       hockey        
thing         government    christ        jpeg          year          
time          state         father        crime         league        
church        turkey        matthew       state         good          


topic 5       topic 6       topic 7       
--------      --------      --------      
file          say           file          
image         kno

Here are my suggestions for topic labels:


0. Interacting with religion.
1. Middle East
2. Christianity
3. Firearms
4. Hockey
5. File systems
6. Basic human actions
7. Programming

Most of these are pretty good with distinct separations. Perhaps topic 6 is a bit of a stretch, but it does seem plausible when just looking at the top 10 words.

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.5 Exploring document topic association
rubric={points:5}

**Your tasks:**
1. Show the document topic assignment of the first five documents from `df`. 
2. Comment on the document topic assignment of the model. 

<div class="alert alert-warning">

Solution_2_5
    
</div>

_Points:_ 5

In [44]:
new_topic_labels = ["Interacting with Religion", "Middle East", "Christianity", "Firearms", "Hockey", "File Systems", "Basic Human Actions", "Programming"]

pd.DataFrame(document_topics, index=df["text"], columns=new_topic_labels).assign(sum=document_topics.sum(axis=1)).head()

,Interacting with Religion,Middle East,Christianity,Firearms,Hockey,File Systems,Basic Human Actions,Programming,sum
text,,,,,,,,,
"You know, I was reading 18 U.S.C. 922 and something just did not make \nsence and I was wondering if someone could help me out.\n\nSay U.S.C. 922 :\n\n(1) Except as provided in paragraph (2), it shall be unlawful for\nany person to transfer or possess a machinegun.\n\n Well I got to looking in my law dictionary and I found that a ""person"" \nmight also be an artificial entity that is created by government \nand has no rights under the federal constitution. So, what I \ndon't understand is how a statute like 922 can be enforced on \nan individual. So someone tell me how my government can tell\nme what I can or cannot possess. Just passing a law \ndoes not make it LAW. Everyone knows that laws are constitional\nuntil it goes to court. So, has it ever gone to court, not\njust your run of the mill ""Ok I had it I am guilty, put me in jail""\n\nHas anyone ever claimed that they had a right to possess and was told\nby the Supreme Court that they didn't have that right?\n\n\n",0.362092,0.002454,0.084233,0.541408,0.002452,0.002454,0.002455,0.002451,1.0
"\n\n\nIt's not a bad question: I don't have any refs that list this algorithm\neither. But thinking about it a bit, it shouldn't be too hard.\n\n1) Take three of the points and find the plane they define as well as\nthe circle that they lie on (you say you have this algorithm already)\n\n2) Find the center of this circle. The line passing through this center\nperpendicular to the plane of the three points passes through the center of\nthe sphere.\n\n3) Repeat with the unused point and two of the original points. This\ngives you two different lines that both pass through the sphere's\norigin. Their interection is the center of the sphere.\n\n4) the radius is easy to compute, it's just the distance from the center to\nany of the original points.\n\nI'll leave the math to you, but this is a workable algorithm. :-)\n\n\nAn alternate method would be to take pairs of points: the plane formed\nby the perpendicular bisector of each line segment pair also contains the\ncenter of the sphere. Three pairs will form three planes, intersecting\nat a point. This might be easier to implement.",0.001763,0.184155,0.001762,0.278485,0.112503,0.417807,0.001763,0.001761,1.0
"\nActuallay I don't, but on the other hand I don't support the idea of having\none newsgroup for every aspect of graphics programming as proposed by Brian,\nin his reply to my original posting.\nI would suggest a looser structure more like a comp.graphics.programmer,\ncomp.graphics.hw_and_sw\nThe reason for making as few groups as possible is for the same reason you\nsay we shouldn't spilt up, not to get to few postings every day.\nI takes to much time to browse through all postings just to find two or \nthree I'm interested in.\n\nI understand and agree when you say you want all aspects of graphics in one\nmeeting. I agree to some extension. I see news as a forum to exchange ideas,\nhelp others or to be helped. I think this is difficult to achive if there\nare so many different things in one meeting.\n\nGood evening netters|-)",0.692948,0.002406,0.075721,0.002407,0.002406,0.219299,0.002409,0.002405,1.0
"The following problem is really bugging me,\nand I would appreciate any help.\n\nI create two windows:\n\nw1 (child to root) with event_mask = ButtonPressMask|KeyPressMask;\nw2 (child to w1) with do_not_propagate_mask = ButtonPressMask|KeyPressMask;\n\n\nKeypress events in w2 are discarded, but ButtonPress events fall through\nto w1, with subwindow set to w2.\n\nFYI, I'm using xnews/olvwm.\n\nAm I doing something fundamentally wrong here?",0.004315,0.004318,0.004313,0.004312,0.004312,0.400758,0.573358,0.004314,1.0
\n\n This is the latest from UPI \n\n Foreign Ministry spokesman Ferhat Ataman told journalists Turkey was\n closing its air space to all flights to and from Armenia and would\n prevent humanitar

From this, we get the following:

Doc 1 = Firearms

Doc 2 = File Systems

Doc 3 = Interacting with Religion

Doc 4 = Basic Human Actions

Doc 5 = Middle East

This is somewhat impressive. Documents 1 and 5 are pretty on the mark in terms of topic assignment. Document 2 is pretty close, but would probably be closer to something like "Math". Documents 3 and 4 are quite off though, so the model isn't perfect.

<!-- END QUESTION -->

<br><br><br><br>

<!-- BEGIN QUESTION -->

## Exercise 3: Short answer questions 
<hr>

rubric={points:6}

1. In lecture 19, we talked about multi-class classification. Comment on how each model in the list below might be handling multiclass classification. Check `scikit-learn` documentation for each of these models when you answer this question.  
    - Decision Tree
    - KNN
    - Random Forest    
    - Logistic Regression
    - SVM RBF
2. What is transfer learning in natural language processing or computer vision? Briefly explain.     
3. In Lecture 19 we briefly discussed how neural networks are sort of like `Pipeline`s, in the sense that they involve multiple sequential transformations of the data, finally resulting in the prediction. Why was this property useful when it came to transfer learning?

<div class="alert alert-warning">

Solution_3
    
</div>

_Points:_ 6

1.
- The decision tree classifier in SKLearn can be used for multi-class classification by default (as stated in the documentation). Decision trees can handle multi-class classification by using a "one versus rest" approach. To do this, there would be one tree made for each class. The classifier would then see if the first tree lands on a "class0" leaf node or a "not class0" leaf node.

- The KNN classifier from SKLearn can also be used for multi-class classification by default (as stated in the documentation). A multi-class KNN classification would work very similar to a binary KNN classification. The only difference would that instead of having points representing 2 classes in our n dimensional space, we would instead have k classes. We would then calculate the distances as normal, and assign a class. This works in a similar way to "one versus rest".
- The Random Forest Classifier in SKLearn can also be used for multi-class classification by default (as stated in the documentation). Given that a random forest is just created from many decision trees, I imagine the multiclassification is handled with many "one versus rest" decision trees.
- Logistic regression can also be used for multi-class classification by default (as stated in the documentation). The Logistic Regression classifier uses the "One versus rest" approach, and it could be implemented by making k binary classifiers. When predicting, the model will run each of the binary classifiers, and whichever classifier ends up with the highest raw score will be produced as the prediction.
- SVM RBF can also be used for multi-class classification by default (as stated in the documentation). The SVM RBF classifier can handle multi-class classification problems using the "one versus rest" or the "one versus one" approach. One possible implementation for the "one versus rest" approach would be to similar to binary classification with SVM RBF. The only difference would that instead of having points representing 2 classes in our n dimensional space, we would instead have k classes. We would then calculate the distances as normal, find support vectors, and assign a class.

2. Transfer learning is the technique that allows both NLP and computer vision developers to download and use a model pre-trained on a certain set of data, then fine tune the model for their specific use case. The benefit of this is that teams can spend lots of time, money and resources creating a model that can generalize fairly well, then other people can simply borrow the model and perform classifications without having to spend millions of dollars.

3. One very beneficial part of the transfer learning pipeline is that we have access to all of the transformations that the pipeline is undergoing. That means we can let the model perform all of the pipeline steps, then swoop in right before the last step (classification), and take the transformed features that the pre-trained model produced. This is a very powerful way to conduct feature selection. Once we have our preprocessed data, we can then pass that data into a classification model of our choice (for example our own linear regression model).

<!-- END QUESTION -->

<br><br><br><br>

**PLEASE READ BEFORE YOU SUBMIT:** 

When you are ready to submit your assignment do the following:

1. Run all cells in your notebook to make sure there are no errors by doing `Kernel -> Restart Kernel and Clear All Outputs` and then `Run -> Run All Cells`. 
2. Notebooks with cell execution numbers out of order or not starting from "1" will have marks deducted. Notebooks without the output displayed may not be graded at all (because we need to see the output in order to grade your work).
3. Upload the assignment using Gradescope's drag and drop tool. Check out this [Gradescope Student Guide](https://lthub.ubc.ca/guides/gradescope-student-guide/) if you need help with Gradescope submission. 
4. Make sure that the plots and output are rendered properly in your submitted file. If the .ipynb file is too big and doesn't render on Gradescope, also upload a pdf or html in addition to the .ipynb so that the TAs can view your submission on Gradescope. 

![](img/eva-well-done.png)